1. Создать Dataset для загрузки данных (sklearn.datasets.fetch_california_housing)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [1]:
import math
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn import datasets, model_selection, preprocessing

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
dataset = datasets.fetch_california_housing()

In [4]:
class HousingDataset():
    def __init__(self, *init_datasets):
        assert all(init_datasets[0].size(0) == init_dataset.size(0) for init_dataset in init_datasets), "Несоотвутствует размерность среди dataset"
        self._base_datasets = init_datasets


    def __len__(self):
        return self._base_datasets[0].size(0)


    def __getitem__(self, idx):
        return tuple(base_dataset[idx] for base_dataset in self._base_datasets)

In [5]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(dataset.data, dataset.target, random_state=13, test_size=0.25)
scale = preprocessing.StandardScaler()
X_train_s = scale.fit_transform(X_train)
X_test_s = scale.transform(X_test)

In [6]:
train_xt = torch.from_numpy(X_train_s.astype(np.float32)).to(device)
train_yt = torch.from_numpy(y_train.astype(np.float32)).to(device)
test_xt = torch.from_numpy(X_test_s.astype(np.float32)).to(device)
test_yt = torch.from_numpy(y_test.astype(np.float32)).to(device)

In [7]:
train_dataset = HousingDataset(train_xt, train_yt)
test_dataset = HousingDataset(test_xt, test_yt)

In [8]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=0, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0, drop_last=True)

In [9]:
class HousingNet(nn.Module):
  def __init__(self) -> None:
      super(HousingNet, self).__init__()
      self.block_1 = nn.Sequential(
          nn.Linear(in_features=8, out_features=200, bias=True),
          nn.Dropout(0.1),
          nn.BatchNorm1d(200),
          nn.ReLU())
      self.block_2 = nn.Sequential(
          nn.Linear(in_features=200, out_features=150, bias=True),
          nn.Dropout(0.1),
          nn.BatchNorm1d(150),
          nn.ReLU())
      self.block_3 = nn.Sequential(
          nn.Linear(in_features=150, out_features=100, bias=True),
          nn.Dropout(0.1),
          nn.BatchNorm1d(100),
          nn.ReLU())
      self.block_4 = nn.Sequential(
          nn.Linear(in_features=100, out_features=80, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(80),
          nn.ReLU())
      self.block_5 = nn.Sequential(
          nn.Linear(in_features=80, out_features=30, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(30),
          nn.ReLU())
      self.predict = nn.Sequential(
          nn.Linear(in_features=30, out_features=1, bias=True),
          nn.BatchNorm1d(1),
          nn.ReLU())
  
  def forward(self, inp):
    out = self.block_1(inp)
    out = self.block_2(out)
    out = self.block_3(out)
    out = self.block_4(out)
    out = self.block_5(out)
    out = self.predict(out)
    return out[:, 0]

In [10]:
def train_loop(num_epochs, train_loader, test_loader, net, optimizer, criterion):
    best_acc = {'train': None, 'test': None}
    for epoch in range(num_epochs):
        running_loss, running_items, running_right = 0.0, 0.0, 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data[0], data[1]
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels).to(device)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_items += len(labels)
            if i % 150 == 0 or (i + 1) == len(train_loader):
                net.eval()
                test_loss, test_running_total, test_loss  = 0.0, 0.0, 0.0
                for y, (out_test, lbl_test) in enumerate(test_loader):
                    test_outputs = net(out_test)
                    test_loss += criterion(test_outputs, lbl_test)
                    test_running_total += len(lbl_test)
                res_loss_train = running_loss / running_items
                res_loss_test = test_loss / test_running_total
                if best_acc['train'] is None or res_loss_train < best_acc['train']:
                    best_acc['train'] = res_loss_train
                if best_acc['test'] is None or res_loss_test < best_acc['test']:
                    best_acc['test'] = res_loss_train
                print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                    f'Step [{i + 1}/{len(train_loader)}]. ' \
                    f'Loss: {res_loss_train:.3f}. '\
                    f'Test acc: {res_loss_test:.3f}.')
                running_loss, running_items = 0.0, 0.0
                net.train()
    return best_acc['train'], best_acc['test']

In [11]:
lr_ = 0.01
criterion = nn.MSELoss()
acc_table = {}

### Adam

In [12]:
net = HousingNet().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=lr_)

In [13]:
acc_train, acc_test = train_loop(10, train_loader, test_loader, net, optimizer, criterion)
acc_table['Adam'] = {'acc_train': acc_train, 'acc_test': acc_test}

Epoch [1/10]. Step [1/241]. Loss: 0.068. Test acc: 0.086.
Epoch [1/10]. Step [151/241]. Loss: 0.025. Test acc: 0.011.
Epoch [1/10]. Step [241/241]. Loss: 0.010. Test acc: 0.015.
Epoch [2/10]. Step [1/241]. Loss: 0.008. Test acc: 0.015.
Epoch [2/10]. Step [151/241]. Loss: 0.008. Test acc: 0.008.
Epoch [2/10]. Step [241/241]. Loss: 0.007. Test acc: 0.015.
Epoch [3/10]. Step [1/241]. Loss: 0.006. Test acc: 0.015.
Epoch [3/10]. Step [151/241]. Loss: 0.007. Test acc: 0.008.
Epoch [3/10]. Step [241/241]. Loss: 0.007. Test acc: 0.016.
Epoch [4/10]. Step [1/241]. Loss: 0.007. Test acc: 0.015.
Epoch [4/10]. Step [151/241]. Loss: 0.007. Test acc: 0.008.
Epoch [4/10]. Step [241/241]. Loss: 0.007. Test acc: 0.013.
Epoch [5/10]. Step [1/241]. Loss: 0.004. Test acc: 0.013.
Epoch [5/10]. Step [151/241]. Loss: 0.007. Test acc: 0.008.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.010.
Epoch [6/10]. Step [1/241]. Loss: 0.006. Test acc: 0.010.
Epoch [6/10]. Step [151/241]. Loss: 0.006. Test acc:

### RMSProp

In [14]:
net = HousingNet().to(device)
optimizer = torch.optim.RMSprop(net.parameters(), lr=lr_)

In [15]:
acc_train, acc_test = train_loop(10, train_loader, test_loader, net, optimizer, criterion)
acc_table['LMSProp'] = {'acc_train': acc_train, 'acc_test': acc_test}

Epoch [1/10]. Step [1/241]. Loss: 0.076. Test acc: 0.100.
Epoch [1/10]. Step [151/241]. Loss: 0.014. Test acc: 0.007.
Epoch [1/10]. Step [241/241]. Loss: 0.008. Test acc: 0.012.
Epoch [2/10]. Step [1/241]. Loss: 0.006. Test acc: 0.012.
Epoch [2/10]. Step [151/241]. Loss: 0.008. Test acc: 0.007.
Epoch [2/10]. Step [241/241]. Loss: 0.007. Test acc: 0.013.
Epoch [3/10]. Step [1/241]. Loss: 0.005. Test acc: 0.013.
Epoch [3/10]. Step [151/241]. Loss: 0.007. Test acc: 0.008.
Epoch [3/10]. Step [241/241]. Loss: 0.007. Test acc: 0.013.
Epoch [4/10]. Step [1/241]. Loss: 0.005. Test acc: 0.012.
Epoch [4/10]. Step [151/241]. Loss: 0.007. Test acc: 0.007.
Epoch [4/10]. Step [241/241]. Loss: 0.007. Test acc: 0.013.
Epoch [5/10]. Step [1/241]. Loss: 0.004. Test acc: 0.012.
Epoch [5/10]. Step [151/241]. Loss: 0.006. Test acc: 0.008.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.013.
Epoch [6/10]. Step [1/241]. Loss: 0.005. Test acc: 0.013.
Epoch [6/10]. Step [151/241]. Loss: 0.006. Test acc:

### SGD

In [16]:
net = HousingNet().to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=lr_)

In [17]:
acc_train, acc_test = train_loop(10, train_loader, test_loader, net, optimizer, criterion)
acc_table['SGD'] = {'acc_train': acc_train, 'acc_test': acc_test}

Epoch [1/10]. Step [1/241]. Loss: 0.071. Test acc: 0.088.
Epoch [1/10]. Step [151/241]. Loss: 0.025. Test acc: 0.010.
Epoch [1/10]. Step [241/241]. Loss: 0.011. Test acc: 0.010.
Epoch [2/10]. Step [1/241]. Loss: 0.011. Test acc: 0.010.
Epoch [2/10]. Step [151/241]. Loss: 0.010. Test acc: 0.009.
Epoch [2/10]. Step [241/241]. Loss: 0.010. Test acc: 0.009.
Epoch [3/10]. Step [1/241]. Loss: 0.010. Test acc: 0.009.
Epoch [3/10]. Step [151/241]. Loss: 0.009. Test acc: 0.008.
Epoch [3/10]. Step [241/241]. Loss: 0.009. Test acc: 0.008.
Epoch [4/10]. Step [1/241]. Loss: 0.008. Test acc: 0.008.
Epoch [4/10]. Step [151/241]. Loss: 0.008. Test acc: 0.007.
Epoch [4/10]. Step [241/241]. Loss: 0.008. Test acc: 0.008.
Epoch [5/10]. Step [1/241]. Loss: 0.006. Test acc: 0.008.
Epoch [5/10]. Step [151/241]. Loss: 0.007. Test acc: 0.006.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.008.
Epoch [6/10]. Step [1/241]. Loss: 0.005. Test acc: 0.008.
Epoch [6/10]. Step [151/241]. Loss: 0.007. Test acc:

### SGD + Momentum

In [18]:
net = HousingNet().to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=lr_, momentum=0.8)

In [19]:
acc_train, acc_test = train_loop(10, train_loader, test_loader, net, optimizer, criterion)
acc_table['SGD+Momentum'] = {'acc_train': acc_train, 'acc_test': acc_test}

Epoch [1/10]. Step [1/241]. Loss: 0.076. Test acc: 0.084.
Epoch [1/10]. Step [151/241]. Loss: 0.015. Test acc: 0.008.
Epoch [1/10]. Step [241/241]. Loss: 0.008. Test acc: 0.009.
Epoch [2/10]. Step [1/241]. Loss: 0.007. Test acc: 0.008.
Epoch [2/10]. Step [151/241]. Loss: 0.007. Test acc: 0.006.
Epoch [2/10]. Step [241/241]. Loss: 0.007. Test acc: 0.009.
Epoch [3/10]. Step [1/241]. Loss: 0.005. Test acc: 0.009.
Epoch [3/10]. Step [151/241]. Loss: 0.007. Test acc: 0.006.
Epoch [3/10]. Step [241/241]. Loss: 0.007. Test acc: 0.008.
Epoch [4/10]. Step [1/241]. Loss: 0.006. Test acc: 0.008.
Epoch [4/10]. Step [151/241]. Loss: 0.007. Test acc: 0.006.
Epoch [4/10]. Step [241/241]. Loss: 0.007. Test acc: 0.010.
Epoch [5/10]. Step [1/241]. Loss: 0.005. Test acc: 0.010.
Epoch [5/10]. Step [151/241]. Loss: 0.006. Test acc: 0.006.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.009.
Epoch [6/10]. Step [1/241]. Loss: 0.004. Test acc: 0.008.
Epoch [6/10]. Step [151/241]. Loss: 0.006. Test acc:

### Результаты

In [20]:
pd.DataFrame(acc_table).transpose()

,acc_train,acc_test
Adam,0.004182,0.006743
LMSProp,0.003902,0.005971
SGD,0.004873,0.005817
SGD+Momentum,0.003830,0.005770
